In [16]:
#coding:utf-8

import random

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn import cross_validation
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt

from nltk import stem
import re
import random

In [17]:
sentences = []

with open("./rt-polaritydata/pos.txt", "r") as f:
    for line in f:
        sentences.append("+1 " + line)

with open("./rt-polaritydata/neg.txt", "r") as f:
    for line in f:
        sentences.append("-1 " + line)

random.shuffle(sentences)

with open("./sentiment.txt", "w") as f:
    for s in sentences:
        f.write(s)

In [18]:
def is_stopword(word):
    return word in count_vectorizer.get_stop_words()

In [19]:
#センテンスを取得
def get_sentence():
    sentences = []

    with open("./rt-polaritydata/pos.txt", "r") as f:
        for line in f:
            sentences.append({'label':1, 'sentence':line})

    with open("./rt-polaritydata/neg.txt", "r") as f:
        for line in f:
            sentences.append({'label':-1, 'sentence':line})

    random.shuffle(sentences)

    return sentences

In [20]:
def create_feature(sentence):
    fuature = []
    words = ""
    stemmer = stem.PorterStemmer()
    sentences = []
    labels = []

    # 素性の作成
    for s in sentence:
        for word in re.compile(r'[,.:;\s]').sub(" ", s['sentence']).split():
            if (not (is_stopword(word))):
                words = words + " " + stemmer.stem(word)
        sentences.append(words)
        labels.append(s['label'])
        words = ""
    fuature.append(sentences)
    fuature.append(labels)
    return fuature

In [21]:
def create_words_weight(vacabulary, coef):
    word_weight = []
    for vocab, weight in zip(vacabulary, coef):
        word_weight.append({'vocab': vocab, 'weight': weight})

    return word_weight

In [22]:
def export_predict(X, Y, lr):
    true_labels = Y
    pre_labels = lr.predict(X)
    prob = lr.predict_proba(X)

    with open("./prob.txt", "w") as f:
        for i, t_label in enumerate(true_labels):
            if t_label==1:
                line = str(t_label) + "  " + str(pre_labels[i]) + "  "+ str(prob[i][1]) +"\n"
            elif t_label==-1:
                line = str(t_label) + "  " + str(pre_labels[i]) + "  "+ str(prob[i][0]) +"\n"

            f.writelines(line)

In [23]:
if __name__ == "__main__":
    count_vectorizer = CountVectorizer(stop_words='english')

    sentence = get_sentence()
    fuature = create_feature(sentence)

    print("fuature:  ",fuature[0][0:20])

fuature:   [" love stori 2002' remark process sweep pictur reinvigor romanc genr", ' movi drag thought heard mysteri voic felt power drawn light -- light exit sign return warn movi 90 minut long life short', ' pretti dicey materi unexpect zig zag help', ' swing fals sentiment unfunni madcap comedi way expect audienc invest central relationship kind marriag true mind', ' jaw-droppingli beauti work upend nearli clich� japanes anim deliv satisfactori carnag', ' battl bug-ey theatr dead-ey matine', ' depress ruthlessli pain deprav movi equival stare open wound', " don't think [kissinger's] guilti crimin activ contemporari statesmen he'd sure make courtroom trial great fun watch", ' stupid annoy', " maneuv skill plot' hot brine -- it' undon soggi contemporari charact actor", ' thought surprisingli affect portrait screwed-up man dare mess power peopl seen eye idealist kid choos champion ultim lose caus', " cho' time priceless", " choos interpret film' end hope optimist think payn darker", ' 

In [26]:
    #単語数のカウント
    feature_vectors = count_vectorizer.fit_transform(fuature[0])
    vocabulary = count_vectorizer.get_feature_names()

    X = feature_vectors.toarray()
    Y = fuature[1]

    print("vocabulary",vocabulary[1000:1050])

vocabulary ['barrel', 'barrett', 'barri', 'barrie', 'barrier', 'barris', 'barrow', 'barry', 'barrymor', 'bart', 'bartlebi', 'bartleby', 'bartlett', 'bas', 'base', 'basebal', 'baseball', 'based', 'basest', 'bash', 'basi', 'basic', 'bask', 'basketbal', 'bass', 'bast', 'bastard', 'bastion', 'bat', 'batch', 'bate', 'bath', 'batho', 'bathroom', 'bathtub', 'batman', 'batter', 'batteri', 'battista', 'battl', 'battlefield', 'bawdi', 'baz', 'bazadona', 'beach', 'beachcomb', 'beacon', 'bead', 'beam', 'bean']


In [27]:
print("X",X)

X [[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]


In [28]:
print(Y[0:100])

[1, -1, 1, -1, 1, -1, -1, 1, -1, 1, 1, 1, 1, 1, -1, 1, 1, -1, 1, -1, 1, 1, 1, -1, 1, 1, -1, 1, -1, 1, 1, -1, 1, -1, 1, -1, 1, 1, -1, 1, -1, 1, 1, 1, -1, -1, 1, 1, 1, -1, -1, -1, 1, -1, -1, 1, 1, 1, -1, 1, -1, 1, -1, -1, -1, 1, -1, -1, 1, -1, 1, -1, 1, -1, -1, 1, -1, 1, 1, -1, 1, 1, 1, 1, -1, 1, 1, 1, -1, 1, 1, 1, -1, 1, -1, 1, -1, -1, 1, -1]


In [30]:
    #学習
    lr = LogisticRegression(C=1000.0)
    lr.fit(X, Y)

    print(lr.get_params)

<bound method BaseEstimator.get_params of LogisticRegression(C=1000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)>


In [31]:
    #文章を与えて確率を予測
    prob = lr.predict_proba(X[0].reshape(1, -1))[0]
    print(prob)

[  7.55079667e-04   9.99244920e-01]


In [33]:
    #単語と重みの紐付け
    word_weight = create_words_weight(vocabulary, lr.coef_[0])

    #TOP10を抽出する
    descend_weight = sorted(word_weight, key=lambda x: x["weight"],reverse=True)
    print("top10", descend_weight[0:11])

top10 [{'vocab': 'badg', 'weight': 20.300929872579335}, {'vocab': 'anatom', 'weight': 19.134890839389183}, {'vocab': 'unsurpass', 'weight': 18.746610509806906}, {'vocab': 'tape', 'weight': 17.731850068991541}, {'vocab': 'taut', 'weight': 17.674107208192794}, {'vocab': 'cloud', 'weight': 17.584325567727856}, {'vocab': 'liber', 'weight': 17.206780720057498}, {'vocab': 'cozi', 'weight': 17.165516349663658}, {'vocab': 'eerili', 'weight': 15.649174340641638}, {'vocab': 'smarter', 'weight': 15.624898836347407}, {'vocab': 'jackie', 'weight': 15.521522162584656}]


In [34]:
    #UNDER10を抽出
    ascend_weight = sorted(word_weight, key=lambda x: x["weight"])
    print("under10", ascend_weight[0:11])

under10 [{'vocab': '85', 'weight': -19.506948923902453}, {'vocab': 'poorli', 'weight': -18.817113735682703}, {'vocab': 'unless', 'weight': -18.063634799969911}, {'vocab': 'picture', 'weight': -17.55862714103672}, {'vocab': 'rosenth', 'weight': -17.283457652570263}, {'vocab': 'devolv', 'weight': -17.171981581336386}, {'vocab': 'languor', 'weight': -16.752574365306874}, {'vocab': 'jumbl', 'weight': -16.641124615154599}, {'vocab': 'witherspoon', 'weight': -16.609216839911056}, {'vocab': 'scooter', 'weight': -16.387293951530467}, {'vocab': 'relic', 'weight': -16.332322014592869}]


In [37]:
y_pre = lr.predict(X)
    
#ラベルと確率をファイル出力
export_predict(X, Y, lr)

In [39]:
    '''
    予測の正解率，正例に関する適合率，再現率，F1スコア
    精度(適合率, precision)：正と予測したデータのうち，実際に正であるものの割合
    再現率 (recall)：実際に正であるもののうち，正であると予測されたものの割
    F値 (F尺度, F-measure)：精度と再現率の調和平均．
    '''
    #正解率
    print('accuracy: ' + str(accuracy_score(Y, y_pre)))
    print(classification_report(Y, y_pre))

accuracy: 0.999249671731
             precision    recall  f1-score   support

         -1       1.00      1.00      1.00      5331
          1       1.00      1.00      1.00      5331

avg / total       1.00      1.00      1.00     10662



In [ ]:
    '''
    5-分割交差検定
    データを5分割して4つを訓練データ、1つをテストデータとして用いる手法
    '''
    
    scores = cross_validation.cross_val_score(lr, X, Y, cv=5, scoring='accuracy')
    print("accuracy:  ", scores)
    scores = cross_validation.cross_val_score(lr, X, Y, cv=5, scoring='precision')
    print("precision:  ", scores)
    scores = cross_validation.cross_val_score(lr, X, Y, cv=5, scoring='recall')
    print("recall:  ", scores)
    scores = cross_validation.cross_val_score(lr, X, Y, cv=5, scoring='f1_weighted')
    print("f1_weighted:  ", scores)

In [ ]:
thresholds = [{1:0.99, -1:0.01},{1:0.95, -1:0.05},{1:0.9, -1:0.1},{1:0.85, -1:0.15},{1:0.8, -1:0.2}
        ,{1:0.75, -1:0.25},{1:0.7, -1:0.3},{1:0.65, -1:0.35},{1:0.6, -1:0.4},{1:0.55, -1:0.45},{1:0.5, -1:0.5}
        ,{1:0.45, -1:0.55},{1:0.4, -1:0.6}, {1:0.35, -1:0.7}, {1:0.3, -1:0.7}, {1:0.25, -1:0.8},{1:0.2, -1:0.8}
        , {1:0.15, -1:0.85},{1:0.1, -1:0.9}, {1:0.05, -1:0.95}, {1:0.01, -1: 0.99}]
    t = [0.99,0.95,0.9,0.85,0.8,0.75,0.7,0.65,0.6,0.55,0.5,0.45,0.4,0.35,0.3,0.25,0.2,0.15,0.1,0.05,0.01]

    precision_rates = []
    recall_rates = []

    for threshold in thresholds:
        scores = cross_validation.cross_val_score(lr, X, Y, cv=5, scoring='precision')
        precision_rates.append(scores[0])
        scores = cross_validation.cross_val_score(lr, X, Y, cv=5, scoring='recall')
        recall_rates.append(scores[0])

In [ ]:
plt.plot(thresholds, precision_rates, label="precision", color="red")
    plt.plot(thresholds, recall_rates, label="recall", color="blue")

    plt.xlabel("threshold")
    plt.ylabel("rate")
    plt.xlim(-0.05, 0.5)
    plt.ylim(0, 1)
    plt.title("Logistic Regression")
    plt.show()